In [1]:
import pandas as pd
from sklearn.metrics import auc
from os import makedirs
from os.path import join
from evaluator import Evaluator, CropOffsetFilter
from training_models import Autoencoder, DenoisingAutoencoder, AutoencoderEnsemble
from monitor_models import Autocovariance, AugmentationCrossCovariances, RepresentationCrossCovariances
from metrics import Identity, Variance, CovarianceWeightedAverage
from directory import RESULTS_DIR, get_history_csv_path

2023-09-11 20:50:46.837738: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 20:50:47.114270: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-11 20:50:48.293308: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64:/usr/local/cuda/lib64:/home/onur/anaconda3/envs/tf/lib/
2023-09-11 20:50:48

In [2]:
EXP_NAME = "exp_test"
TRAIN_DATASETS = ["instance", "stead"]
TEST_DATASETS = ["instance", "stead"]
SPLITS = range(0, 5)
NUM_EPOCHS = 20
FILTERS = [CropOffsetFilter(3.0, 3.0, 30.0, 100.0)]
OUTPUT_FILE_NAME = "summary.csv"

In [3]:
def calculate_roc_aucs(training_model_class, monitoring_model_class, train_dataset, test_dataset, split, method_params, metric):
    evaluator = Evaluator(EXP_NAME, training_model_class, monitoring_model_class, 
                          train_dataset, test_dataset, FILTERS, range(NUM_EPOCHS), split, method_params, metric)
                    
    roc_vectors = evaluator.get_roc_vectors()
    roc_aucs = []

    for epoch in range(NUM_EPOCHS):
        roc_aucs.append(auc(roc_vectors[epoch]["fpr"], roc_vectors[epoch]["tpr"]))

    return roc_aucs

def get_validation_losses(training_model_class, train_dataset, split):
    model = training_model_class()
    df_history = pd.read_csv(get_history_csv_path(EXP_NAME, model.name, train_dataset, split))
    return list(df_history.loc[:, "val_loss"].values)

def model_summary(training_model_class, monitoring_model_class, method_params, metric):
    df_evaluations = []
    for train_dataset in TRAIN_DATASETS:
        for test_dataset in TEST_DATASETS:
            for split in SPLITS:
                roc_aucs = calculate_roc_aucs(training_model_class, monitoring_model_class, train_dataset, test_dataset, split, method_params, metric)
                val_loss = get_validation_losses(training_model_class, train_dataset, split)
                df_evaluations.append(pd.DataFrame(
                                    {"Training dataset": [train_dataset]*NUM_EPOCHS, 
                                    "Testing dataset": [test_dataset]*NUM_EPOCHS, 
                                    "Split": [split]*NUM_EPOCHS, 
                                    "Epoch": list(range(NUM_EPOCHS)), 
                                    "Validation loss": val_loss, 
                                    "ROC-AUC":roc_aucs}))

    df = pd.concat(df_evaluations, ignore_index=True)

    df.loc[:, "Training model"] = training_model_class().name
    df.loc[:, "Monitoring model"] = monitoring_model_class().name

    return df

def grand_summary():
    df_autoencoder_autocovariance = model_summary(Autoencoder, Autocovariance, 
                                                  {}, 
                                                  CovarianceWeightedAverage(input_param="fcov"))
    
    df_denoisingae_autocovariance = model_summary(DenoisingAutoencoder, Autocovariance, 
                                                  {}, 
                                                  CovarianceWeightedAverage(input_param="fcov"))
    
    df_autoencoder_augmentation_crosscov = model_summary(Autoencoder, AugmentationCrossCovariances, 
                                                         {"augmentations": 5, "std": 0.15, "knots": 4}, 
                                                         CovarianceWeightedAverage(input_param="fcov"))
    
    df_denoisingae_augmentation_crosscov = model_summary(DenoisingAutoencoder, AugmentationCrossCovariances, 
                                                         {"augmentations": 5, "std": 0.15, "knots": 4}, 
                                                         CovarianceWeightedAverage(input_param="fcov"))
    
    df_autoencoder_ensemble_representation_crosscov = model_summary(AutoencoderEnsemble, RepresentationCrossCovariances, 
                                                                    {}, 
                                                                    CovarianceWeightedAverage(input_param="fcov"))

    df_grand_summary = pd.concat(
        [
            df_autoencoder_autocovariance,
            df_denoisingae_autocovariance,
            df_autoencoder_augmentation_crosscov,
            df_denoisingae_augmentation_crosscov,
            df_autoencoder_ensemble_representation_crosscov,
        ],
        ignore_index=True
    )

    return df_grand_summary

In [4]:
df = grand_summary()

2023-09-11 20:50:49.266368: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-11 20:50:49.568191: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-11 20:50:49.568342: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-11 20:50:49.568889: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuil

989/989 [==============================] - 49s 48ms/step


/home/onur/Desktop/GitRepos/LatentCovarianceBasedSeismicEventDetection/evaluator.py:204: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._get_meta_file_path())
/home/onur/Desktop/GitRepos/LatentCovarianceBasedSeismicEventDetection/kfold_environment.py:390: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(metadata_csv)


987/987 [==============================] - 49s 48ms/step


/home/onur/Desktop/GitRepos/LatentCovarianceBasedSeismicEventDetection/evaluator.py:204: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._get_meta_file_path())
/home/onur/Desktop/GitRepos/LatentCovarianceBasedSeismicEventDetection/kfold_environment.py:428: DtypeWarning: Columns (23,24,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  eq_metadata = pd.read_csv(eq_metadata_csv)


...

/home/onur/Desktop/GitRepos/LatentCovarianceBasedSeismicEventDetection/kfold_environment.py:428: DtypeWarning: Columns (23,24,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  eq_metadata = pd.read_csv(eq_metadata_csv)


1007/1007 [==============================] - 46s 44ms/step


/home/onur/Desktop/GitRepos/LatentCovarianceBasedSeismicEventDetection/kfold_environment.py:428: DtypeWarning: Columns (23,24,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  eq_metadata = pd.read_csv(eq_metadata_csv)


1026/1026 [==============================] - 47s 44ms/step


/home/onur/Desktop/GitRepos/LatentCovarianceBasedSeismicEventDetection/kfold_environment.py:390: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(metadata_csv)
/home/onur/Desktop/GitRepos/LatentCovarianceBasedSeismicEventDetection/evaluator.py:204: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._get_meta_file_path())
/home/onur/Desktop/GitRepos/LatentCovarianceBasedSeismicEventDetection/kfold_environment.py:390: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(metadata_csv)
/home/onur/Desktop/GitRepos/LatentCovarianceBasedSeismicEventDetection/evaluator.py:204: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._get_meta_file_path())
/home/onur/Desktop/GitRepos/LatentCovarianceBase

In [ ]:
makedirs(join(RESULTS_DIR, EXP_NAME), exist_ok=True)


In [ ]:
df.to_csv(join(RESULTS_DIR, EXP_NAME, OUTPUT_FILE_NAME))